1. Import Necessary Libraries

In [60]:
# Install required packages
!pip install sentence-transformers
!pip install transformers
!pip install scikit-learn
!pip install datasets
!pip install tensorflow
!pip install focal-loss

# Import necessary libraries
import pandas as pd
from sentence_transformers import SentenceTransformer, losses, InputExample, models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import tensorflow as tf
from datasets import Dataset
from focal_loss import BinaryFocalLoss

In [61]:
!pip install comet_ml

In [62]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
experiment = Experiment(
  api_key="j7D2qTwhmdLpvtUg9wlAC5nfD",
  project_name="general",
  workspace="omjamil-microsoft-com"
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: keras, tensorflow, torch, sklearn.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : novel_offset_2539
COMET INFO:     url                   : https://www.comet.com/omjamil-microsoft-com/general/636fa7b278b7437cae5bd56aa29b9c4d
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: keras,

#Load the data

In [63]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
data = pd.read_csv('/content/drive/My Drive/Project 266 files/training_data_modified_final.csv')

Remove Unclassiable Rows and Keep only relevant columns

In [65]:
data_cleaned = data[data['classification_raw'] != 'Unclassifiable']
data_cleaned = data_cleaned[['sentence', 'objectivity_classification']]

In [66]:
data_cleaned.value_counts('objectivity_classification')

,count
objectivity_classification,
True,12967
False,8126


In [67]:
# Convert 'objectivity_classification' to boolean: True for Objective, False for Subjective
data_cleaned['objectivity_classification'] = data_cleaned['objectivity_classification'].apply(lambda x: x == True)

In [68]:
# Split the data into training and testing sets
X = data_cleaned['sentence'].tolist()
y = data_cleaned['objectivity_classification'].tolist()
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [69]:
X_train[:5]

['in december 2007 the elder kim received a personal letter from thenpresident george w bush asking that the country dismantle its nuclear weapons program after north korea pledged during talks in geneva to do so',
 'and they began to identify organizations that were sympathetic to this holistic approach',
 'colleagues this morning ocasiocortez wrote on twitter where she has 24 million followers',
 'even with the current expansion nearly 10 years old the us economy is showing resilience',
 'according to media reports an airstrike hit a clinic treating some of the victims a few hours later']

In [70]:
y_train[:5]

[True, True, True, True, True]

Baseline: Load pretrained sentence transformer. Encode sentences. Train logistic Classifier predict and evaluate



In [71]:
# Load a pretrained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [72]:
# encode the sentences into embeddings
x_train_embeddings = model.encode(X_train)
x_test_embeddings = model.encode(X_test)
x_val_embeddings = model.encode(X_val)

In [73]:
#Train a logistic regression model on the embeddings
Classifier = LogisticRegression()
Classifier.fit(x_train_embeddings, y_train)


LogisticRegression()

In [74]:
# make predictions on the test data set
y_pred = Classifier.predict(x_test_embeddings)

In [75]:
# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [76]:
# Print the evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.8601564351742119
Precision: 0.864954128440367
Recall: 0.9139201240791004
F1 Score: 0.888763197586727


LET'S START FINE-TUNING

In [269]:
# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [270]:
# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 128


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed
    into a pre-trained transformer model.

    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""

    input_ids = []
    attention_mask = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=MAX_LENGTH,
                                             padding='max_length', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])

    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)


# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train)

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_val)

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test)

In [271]:
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2

# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT,
                          attention_dropout=DISTILBERT_ATT_DROPOUT,
                          output_hidden_states=True)

# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


ADD TWO NEW CNN LAYERS AS A NEW CLASSIFICATION HEAD

In [272]:
def extract_cls_token(tensor):
    """Extract the [CLS] token from the last hidden state tensor."""
    return tensor[:, 0, :]


In [273]:
MAX_LENGTH = 128
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42

def build_model(transformer, max_length=MAX_LENGTH, lr = LEARNING_RATE, dropout = LAYER_DROPOUT, dense_units = (256,32)):
    """
    Template for building a model off of the BERT or DistilBERT architecture
    for a binary classification task.

    Input:
      - transformer:  a base Hugging Face transformer model object (BERT or DistilBERT)
                      with no added classification head attached.
      - max_length:   integer controlling the maximum number of encoded tokens
                      in a given sequence.

    Output:
      - model:        a compiled tf.keras.Model with added classification layers
                      on top of the base pre-trained model architecture.
    """

    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE)

    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,),
                                            name='input_ids',
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,),
                                                  name='input_attention',
                                                  dtype='int32')

    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]

    # We only care about DistilBERT's output for the [CLS] token,
    # which is located at index 0 of every encoded sequence.
    # Splicing out the [CLS] tokens gives us 2D data.
    cls_token = tf.keras.layers.Lambda(extract_cls_token, name='extract_cls_token')(last_hidden_state)

    x = tf.keras.layers.Dense(dense_units[0], activation='relu', kernel_initializer='he_normal')(cls_token)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(dense_units[1], activation='relu', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Dropout(dropout)(x)

    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(1,
                                   activation='sigmoid',
                                   kernel_initializer=weight_initializer,
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(x)

    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)

    # Compile the model
    model.compile(tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=BinaryFocalLoss(gamma=2),
                  metrics=['accuracy'])

    return model

In [274]:
#Already tested all combinations
# learning_rates = [1e-5, 5e-5, 1e-4]
# dropout_rates = [0.1, 0.2, 0.3]
# dense_units = [(256, 32), (128, 64), (64, 32)]

# EPOCHS = 6
# BATCH_SIZE = 64
# NUM_STEPS = len(X_train) // BATCH_SIZE
# y_train_array = np.array(y_train)
# y_val_array = np.array(y_val)

# # Train the model
# for lr in learning_rates:
#     for dropout in dropout_rates:
#         for units in dense_units:
#             experiment.set_name(f"LR_{lr}_Dropout_{dropout}_Units_{units}")
#             print(f"LR_{lr}_Dropout_{dropout}_Units_{units}")
#             model = build_model(distilBERT, lr=lr, dropout=dropout, dense_units=units)
#             model.fit(
#                 x = [X_train_ids, X_train_attention],
#                 y = y_train_array,
#                 epochs = EPOCHS,
#                 batch_size = BATCH_SIZE,
#                 steps_per_epoch = NUM_STEPS,
#                 validation_data = ([X_valid_ids, X_valid_attention], y_val_array),
#                 verbose=2)
#             experiment.end()

In [275]:
# based on the grid search will choose learning rate of 0.0001, dropout 0.3, and 256, 32 as the CNN layer number
EPOCHS = 6
BATCH_SIZE = 64
NUM_STEPS = len(X_train) // BATCH_SIZE
y_train_array = np.array(y_train)
y_val_array = np.array(y_val)

# Train the model

model = build_model(distilBERT, lr=5e-5, dropout=0.3, dense_units=(256,32))
model.fit(
  x = [X_train_ids, X_train_attention],
  y = y_train_array,
  epochs = EPOCHS,
  batch_size = BATCH_SIZE,
  steps_per_epoch = NUM_STEPS,
  validation_data = ([X_valid_ids, X_valid_attention], y_val_array),
  verbose=2)

Epoch 1/6
197/197 - 36s - loss: 0.1729 - accuracy: 0.6357 - val_loss: 0.1225 - val_accuracy: 0.8097 - 36s/epoch - 184ms/step
Epoch 2/6
197/197 - 28s - loss: 0.1307 - accuracy: 0.7564 - val_loss: 0.0992 - val_accuracy: 0.8367 - 28s/epoch - 140ms/step
Epoch 3/6
197/197 - 28s - loss: 0.1144 - accuracy: 0.8018 - val_loss: 0.0900 - val_accuracy: 0.8559 - 28s/epoch - 140ms/step
Epoch 4/6
197/197 - 28s - loss: 0.1059 - accuracy: 0.8225 - val_loss: 0.0839 - val_accuracy: 0.8613 - 28s/epoch - 141ms/step
Epoch 5/6
197/197 - 28s - loss: 0.1002 - accuracy: 0.8345 - val_loss: 0.0826 - val_accuracy: 0.8632 - 28s/epoch - 140ms/step
Epoch 6/6
197/197 - 28s - loss: 0.0957 - accuracy: 0.8419 - val_loss: 0.0820 - val_accuracy: 0.8656 - 28s/epoch - 140ms/step


In [276]:
FT_EPOCHS = 4
BATCH_SIZE = 64
NUM_STEPS = len(X_train) // BATCH_SIZE

initial_weights = model.get_weights()

# Define the learning rates to search over, including 1e-6
learning_rates = [2e-5]

# Unfreeze distilBERT layers and make available for training
for layer in distilBERT.layers:
    layer.trainable = True

for lr in learning_rates:
  print(f"\nTraining with learning rate: {lr}\n")

  # Reload the initial weights
  model.set_weights(initial_weights)

  # Recompile model after unfreezing
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss=BinaryFocalLoss(gamma=2),
                metrics=['accuracy'])

  # Train the model
  train_history2 = model.fit(
      x = [X_train_ids, X_train_attention],
      y =  y_train_array,
      epochs = FT_EPOCHS,
      batch_size = BATCH_SIZE,
      steps_per_epoch = NUM_STEPS,
      validation_data = ([X_valid_ids, X_valid_attention], y_val_array),
      verbose=2
  )


Training with learning rate: 2e-05

Epoch 1/4
197/197 - 84s - loss: 0.0775 - accuracy: 0.8765 - val_loss: 0.0588 - val_accuracy: 0.9040 - 84s/epoch - 425ms/step
Epoch 2/4
197/197 - 65s - loss: 0.0601 - accuracy: 0.9041 - val_loss: 0.0589 - val_accuracy: 0.9076 - 65s/epoch - 332ms/step
Epoch 3/4
197/197 - 65s - loss: 0.0444 - accuracy: 0.9311 - val_loss: 0.0615 - val_accuracy: 0.9104 - 65s/epoch - 332ms/step
Epoch 4/4
197/197 - 65s - loss: 0.0320 - accuracy: 0.9525 - val_loss: 0.0816 - val_accuracy: 0.9050 - 65s/epoch - 332ms/step


In [277]:
# Function to predict subjectivity of a sentence
def predict_subjectivity(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='tf', max_length=MAX_LENGTH, padding='max_length', truncation=True)
    # Get input_ids and attention_mask
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    # Make predictions
    predictions = model.predict([input_ids, attention_mask])
    # The output itself is the predicted probability (sigmoid output)
    probability = predictions[0][0]
    return probability < 0.5  # Assuming 1 is Objective and 0 is subjective

In [278]:
# Function to predict subjectivity of a sentence
def predict_subjectivity(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='tf', max_length=MAX_LENGTH, padding='max_length', truncation=True)
    # Get input_ids and attention_mask
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    # Make predictions
    predictions = model.predict([input_ids, attention_mask])
    # The output itself is the predicted probability (sigmoid output)
    probability = predictions[0][0]
    print(probability)
    return probability < 0.5  # Assuming 1 is Objective and 0 is Subjective

# Function to score an article based on subjectivity
def score_article(article_text):
    # Split the article into sentences
    sentences = article_text.split('.')

    total_classifiable_sentences = 0
    subjective_count = 0
    too_short_to_classify = 0

    for sentence in sentences:
        # Remove leading and trailing whitespace
        sentence = sentence.strip()
        # Count the words in the sentence
        word_count = len(sentence.split())

        if word_count <= 6:
            too_short_to_classify += 1
        else:
            total_classifiable_sentences += 1
            if predict_subjectivity(sentence):
                subjective_count += 1

    if total_classifiable_sentences == 0:
        # Avoid division by zero if no sentences are classified
        P = 0
    else:
        # Calculate percentage of subjective sentences
        P = (subjective_count / total_classifiable_sentences) * 100

    # Assign a score based on the percentage of subjective sentences
    if P < 7.69:
        score = 5
        display_message = "Extremely Objective"
    elif 7.69 <= P < 15.97:
        score = 4
        display_message = "Mostly Objective"
    elif 15.97 <= P < 22.73:
        score = 3
        display_message = "Moderately Objective"
    elif 22.73 <= P < 30:
        score = 2
        display_message = "Moderately Subjective"
    elif 30 <= P < 40:
        score = 1
        display_message = "Mostly Subjective"
    else:
        score = 0
        display_message = "Extremely Subjective"
    return {
        'total_classifiable_sentences': total_classifiable_sentences,
        'subjective_count': subjective_count,
        'too_short_to_classify': too_short_to_classify,
        'subjectivity_percentage': P,
        'score': score,
        'displayMessage': display_message
    }


In [279]:
# Example usage
article_text = """
oped contributor cassius clay born in 1942 was the grandson of a slave in the united states of his boyhood and young manhood the role of the black athlete particularly the black boxer was a forced selfeffacement. respected publications including the new york times continued to print the slave name cassius clay for years. sentenced to five years imprisonment for his refusal to comply with the draft ali stood his ground he did not serve time but was fined 10000 and his boxing license was revoked so that he could not continue his professional career in the very prime of that career.
"""

In [280]:
result = score_article(article_text)
print(f"Total Classifiable Sentences: {result['total_classifiable_sentences']}")
print(f"Subjective Sentences: {result['subjective_count']}")
print(f"Subjectivity Percentage: {result['subjectivity_percentage']:.2f}%")
print(f"Score: {result['score']} ({result['displayMessage']})")

1/1 [==============================] - 2s 2s/step
0.78940415
1/1 [==============================] - 0s 30ms/step
0.99114054
1/1 [==============================] - 0s 31ms/step
0.85229105
Total Classifiable Sentences: 3
Subjective Sentences: 0
Subjectivity Percentage: 0.00%
Score: 5 (Extremely Objective)


In [281]:
import pandas as pd

def process_summaries_for_subjectivity(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Initialize a list to store the subjectivity scores
    subjectivity_scores = []

    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        summary_text = row['summary']
        # Score the summary
        score_result = score_article(summary_text)
        # Add the subjectivity score to the list
        subjectivity_scores.append(score_result['score'])

    # Add the subjectivity scores as a new column in the DataFrame
    df['subjectivity_score'] = subjectivity_scores

    # Return the updated DataFrame
    return df

In [282]:
# Example usage
file_path = '/content/drive/My Drive/Project 266 files/test_pegasus_fixed.csv'
output_df = process_summaries_for_subjectivity(file_path)

1/1 [==============================] - 0s 31ms/step
0.9912203
1/1 [==============================] - 0s 31ms/step
0.9865052
1/1 [==============================] - 0s 30ms/step
0.76007295
1/1 [==============================] - 0s 30ms/step
0.78940415
1/1 [==============================] - 0s 32ms/step
0.99114054
1/1 [==============================] - 0s 31ms/step
0.85229105
1/1 [==============================] - 0s 30ms/step
0.84586126
1/1 [==============================] - 0s 31ms/step
0.96417457
1/1 [==============================] - 0s 32ms/step
0.55508834
1/1 [==============================] - 0s 31ms/step
0.99861336
1/1 [==============================] - 0s 30ms/step
0.5041425
1/1 [==============================] - 0s 31ms/step
0.9955166
1/1 [==============================] - 0s 33ms/step
0.9957283
1/1 [==============================] - 0s 30ms/step
0.9963846
1/1 [==============================] - 0s 31ms/step
0.9843301
1/1 [==============================] - 0s 29ms/step
0.9506762


In [283]:
output_df.head()

,Unnamed: 0,title,sentence,input_text,article,publication,section,token_count,summary,rouge_scores,rouge1_f1,rouge2_f1,rougeL_f1,subjectivity_score
0,0,John Bolton says he is prepared to testify in ...,['cnnformer white house national security advi...,cnnformer white house national security advise...,(CNN)Former White House national security adv...,CNN,politics,1092,romnnformer white house national security advi...,"{'rouge1': Score(precision=0.9905660377358491,...",0.128049,0.122100,0.128049,5
1,1,Opinion | Muhammad Ali: Never the White Man’s ...,['oped contributor cassius clay born in 1942 w...,oped contributor cassius clay born in 1942 was...,"Op-Ed Contributor CASSIUS CLAY, born in 1942, ...",The New York Times,opinion,225,oped contributor cassius clay born in 1942 was...,"{'rouge1': Score(precision=0.9705882352941176,...",0.199597,0.193939,0.199597,5
2,2,"Opinion | No, Settlers Don’t Control Israeli P...",['shmuel rosner jerusalem a government that i...,shmuel rosner jerusalem a government that is ...,Shmuel Rosner JERUSALEM — A government that is...,The New York Times,opinion,389,shmuel rosner jerusalem a government that is r...,"{'rouge1': Score(precision=0.9895833333333334,...",0.184109,0.174757,0.184109,5
3,3,Mexico president says 'doing well' on migratio...,['mexico city reuters mexicos president said ...,mexico city reuters mexicos president said on...,MEXICO CITY (Reuters) - Mexico’s president sai...,Reuters,World News,410,mexico city reuters mexicos president said on ...,"{'rouge1': Score(precision=0.9818181818181818,...",0.425197,0.403162,0.425197,5
4,4,Kathleen Kane: Pennsylvania attorney general r...,['cnnpennsylvania attorney general kathleen ka...,cnnpennsylvania attorney general kathleen kane...,(CNN)Pennsylvania Attorney General Kathleen K...,CNN,politics,246,cnnpennsylvania attorney general kathleen kane...,"{'rouge1': Score(precision=0.9894736842105263,...",0.250667,0.243316,0.250667,5


In [284]:
output_df.to_csv('/content/drive/My Drive/Project 266 files/test_pegasus_fixed_and_classified.csv', index=False)

In [285]:
output_df['subjectivity_score'].value_counts()

,count
subjectivity_score,
5,59
2,1


In [286]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 input_attention (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 tf_distil_bert_model_3 (TF  TFBaseModelOutput(last_hid   6636288   ['input_ids[0][0]',           
 DistilBertModel)            den_state=(None, 128, 768)   0          'input_attention[0][0]']     
                             , hidden_states=((None, 12                                     

In [287]:
# Save the model in HDF5 format
model.save('/content/drive/My Drive/Project 266 files/fine_tuned_model.keras')

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
